# Credit Portfolio Risk Modelling – Data Preparation

This notebook performs:

- Data loading
- Loan status filtering
- Target variable definition (Default)
- Removal of leakage variables
- Basic cleaning and preprocessing

Dataset: Lending Club Loan Data


In [1]:
## importing the required libraries 
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
%matplotlib inline

In [2]:
%pwd


'C:\\Users\\amit_\\Github\\credit-portfolio-vasicek-simulation\\notebooks'

In [3]:
df = pd.read_csv(r'C:\\Users\\amit_\\Github\\credit-portfolio-vasicek-simulation\\data\\raw\\loan_data.csv', index_col = 0, low_memory=False)

## Dataset Overview

The Lending Club dataset contains retail loan-level information including:

- Borrower characteristics
- Loan structure details
- Repayment and recovery information
- Loan status

This step focuses on:

1. Inspecting dataset structure
2. Understanding loan status distribution
3. Preparing data for PD modelling


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 466285 entries, 0 to 466284
Data columns (total 74 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           466285 non-null  int64  
 1   member_id                    466285 non-null  int64  
 2   loan_amnt                    466285 non-null  int64  
 3   funded_amnt                  466285 non-null  int64  
 4   funded_amnt_inv              466285 non-null  float64
 5   term                         466285 non-null  object 
 6   int_rate                     466285 non-null  float64
 7   installment                  466285 non-null  float64
 8   grade                        466285 non-null  object 
 9   sub_grade                    466285 non-null  object 
 10  emp_title                    438697 non-null  object 
 11  emp_length                   445277 non-null  object 
 12  home_ownership               466285 non-null  object 
 13  annu

## Initial Data Type Observations

After inspecting the dataset using df.info(), the following columns require type correction or cleaning:

- `term` → currently object; should be numeric (loan term in months)
- `emp_length` → object; requires cleaning and numeric mapping
- `pymnt_plan` → categorical (binary)
- `earliest_cr_line` → object; should be converted to datetime
- `issue_d` → object; should be converted to datetime

These transformations are necessary for proper modelling and feature engineering.


## Filtering Resolved Loans

For Probability of Default (PD) modelling, only loans with final outcomes are considered.

Included:
- Fully Paid
- Charged Off

Excluded:
- Current (active loans)

This ensures that the target variable represents observed default behaviour.


In [5]:
df['loan_status'].value_counts()

loan_status
Current                                                224226
Fully Paid                                             184739
Charged Off                                             42475
Late (31-120 days)                                       6900
In Grace Period                                          3146
Does not meet the credit policy. Status:Fully Paid       1988
Late (16-30 days)                                        1218
Default                                                   832
Does not meet the credit policy. Status:Charged Off       761
Name: count, dtype: int64

## Defining Modelling Population

For PD modelling, only loans with resolved outcomes are included.

Default = 
- Charged Off
- Default
- Does not meet credit policy: Status: Charged Off

Non-default =
- Fully Paid
- Does not meet credit policy: Status: Fully Paid

Unresolved loan states (Current, Late, Grace Period) are excluded.
Also, creating a loan_data dataframe a copy of df to protect the raw data

In [6]:
default_status = [
    'Charged Off',
    'Default',
    'Does not meet the credit policy. Status:Charged Off'
]

non_default_status = [
    'Fully Paid',
    'Does not meet the credit policy. Status:Fully Paid'
]
loan_data = df[df['loan_status'].isin(default_status + non_default_status)].copy()


In [7]:
loan_data['loan_status'].value_counts()/loan_data['loan_status'].count()

loan_status
Fully Paid                                             0.800446
Charged Off                                            0.184038
Does not meet the credit policy. Status:Fully Paid     0.008614
Default                                                0.003605
Does not meet the credit policy. Status:Charged Off    0.003297
Name: count, dtype: float64

In [8]:
loan_data['default'] = np.where(
    loan_data['loan_status'].isin(default_status),
    1,
    0
)


In [9]:
loan_data.shape
loan_data['default'].mean()
loan_data['default'].value_counts()


default
0    186727
1     44068
Name: count, dtype: int64

In [10]:
loan_data.dtypes.value_counts()


float64    46
object     22
int64       6
int32       1
Name: count, dtype: int64

### Inspecting Object Variables

Before transforming object variables, we need to understand their nature.

Object columns can represent:

Numeric values stored as text (e.g., "36 months")

Ordered categories (e.g., grade, emp_length)

Nominal categories (e.g., home_ownership)

High-cardinality categorical variables (e.g., purpose)

Irrelevant identifiers (e.g., id, url)

In [11]:
loan_data.select_dtypes(include = 'object').columns

Index(['term', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'verification_status', 'issue_d', 'loan_status',
       'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'earliest_cr_line', 'initial_list_status', 'last_pymnt_d',
       'next_pymnt_d', 'last_credit_pull_d', 'application_type'],
      dtype='object')

## Transforming Object Variables to Numeric Format

In [12]:
loan_data['term'].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [13]:
loan_data['term_month'] = loan_data['term'].str.replace(r'[^0-9]','',regex=True)
loan_data['term_month'] = pd.to_numeric(loan_data['term_month'])

In [14]:
type(loan_data['term_month'][0])

numpy.int64

In [15]:
loan_data[['term', 'term_month']].head()


,term,term_month
0,36 months,36
1,60 months,60
2,36 months,36
3,36 months,36
5,36 months,36


In [16]:
loan_data['term_month'].isna().sum()


0

In [17]:
loan_data['emp_length'].unique()


array(['10+ years', '< 1 year', '3 years', '9 years', '4 years',
       '5 years', '1 year', '6 years', '2 years', '7 years', '8 years',
       nan], dtype=object)

In [18]:
loan_data['emp_length_years'] = loan_data['emp_length'].str.strip()
loan_data['emp_length_years'] = loan_data['emp_length_years'].replace('< 1 year','0')
loan_data['emp_length_years'] = loan_data['emp_length_years'].str.replace(r'[^0-9]','',regex = True)
loan_data['emp_length_years'] = pd.to_numeric(loan_data['emp_length_years'])


In [19]:
type(loan_data['emp_length_years'][0])

numpy.float64

In [20]:
loan_data[['emp_length','emp_length_years']].head(10)

,emp_length,emp_length_years
0,10+ years,10.0
1,< 1 year,0.0
2,10+ years,10.0
3,10+ years,10.0
5,3 years,3.0
7,9 years,9.0
8,4 years,4.0
9,< 1 year,0.0
10,5 years,5.0
11,10+ years,10.0


In [22]:
type(loan_data['earliest_cr_line'][0])

str

## Converting earliest_cr_line to Datetime Format

The earliest_cr_line variable represents the borrower’s earliest reported credit line.
Originally, this column is stored as a string (e.g., "Jan-85"), so it must be converted into a proper datetime format before performing any time-based calculations.

We convert it using pd.to_datetime() with the appropriate format (%b-%y) to correctly interpret the month and year.

This allows us to:

Perform date arithmetic

Derive account age

Identify inconsistencies or anomalies in reported dates

In [26]:
loan_data['earliest_cr_line_date'] = pd.to_datetime(loan_data['earliest_cr_line'],format ='%b-%y')

In [27]:
type(loan_data['earliest_cr_line_date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [28]:
reference_date = pd.to_datetime('2026-01-01')

## Creating months_since_earliest_cr_line

To measure the borrower’s credit history length, we calculate the number of months between a fixed reference date and the borrower’s earliest credit line date.

We use a reference date (2026-01-01) and compute:

Months since earliest credit line =
(Year difference × 12) + Month difference

This creates a continuous numeric variable representing credit history length in months — an important predictor in credit risk modeling.

Longer credit history generally indicates greater borrower stability and richer behavioral information.

In [29]:
loan_data['months_since_earliest_cr_line'] = ((reference_date.year - loan_data['earliest_cr_line_date'].dt.year) * 12 + 
                                            (reference_date.month - loan_data['earliest_cr_line_date'].dt.month))

In [30]:
loan_data['months_since_earliest_cr_line'].describe()

count    230766.000000
mean        333.890582
std          89.732723
min        -515.000000
25%         280.000000
50%         320.000000
75%         376.000000
max         684.000000
Name: months_since_earliest_cr_line, dtype: float64

## Identifying Data Anomalies (Negative Values)

During exploratory checks, we observed negative values in months_since_earliest_cr_line.

Negative values imply that some credit line dates are in the future relative to the reference date — which is logically impossible.

This typically happens due to century interpretation issues (e.g., 2068 instead of 1968 when parsing two-digit years).

We investigate these anomalies to ensure data integrity before modeling.

In [32]:
(loan_data.loc[loan_data['months_since_earliest_cr_line']< 0 , ['months_since_earliest_cr_line','earliest_cr_line_date']]).head(5)

,months_since_earliest_cr_line,earliest_cr_line_date
1580,-440.0,2062-09-01
1770,-512.0,2068-09-01
3282,-500.0,2067-09-01
3359,-469.0,2065-02-01
3413,-497.0,2067-06-01


## Correcting Century Misinterpretation

To correct the issue, we adjust any year that is greater than the reference year by subtracting 100 years.

This effectively converts misinterpreted future dates (e.g., 2068) into their correct historical values (e.g., 1968).

After correction, we recompute months_since_earliest_cr_line to ensure all values are positive and logically consistent.

This step ensures accurate measurement of borrower credit history.

In [33]:
loan_data['earliest_cr_line_date'] = loan_data['earliest_cr_line_date'].apply( lambda x : x.replace(year = x.year - 100)
                                                                               if x.year > reference_date .year 
                                                                               else x )

In [34]:
loan_data['months_since_earliest_cr_line'] = ((reference_date.year - loan_data['earliest_cr_line_date'].dt.year) * 12 + 
                                            (reference_date.month - loan_data['earliest_cr_line_date'].dt.month))

In [35]:
loan_data['months_since_earliest_cr_line'].describe()

count    230766.000000
mean        336.480218
std          83.446374
min         170.000000
25%         280.000000
50%         321.000000
75%         377.000000
max         960.000000
Name: months_since_earliest_cr_line, dtype: float64